In [2]:
%%writefile plug_ai.slurm
#!/bin/bash
#SBATCH --job-name=plug_ai
#SBATCH --output=plug_ai.out
#SBATCH --error=plug_ai.out
#SBATCH --gres=gpu:1
#SBATCH --ntasks-per-node=1
#SBATCH --nodes=1
#SBATCH --hint=nomultithread
#SBATCH --time=00:50:00
#SBATCH --qos=qos_gpu-dev
#SBATCH --cpus-per-task=10
#SBATCH --account=sos@v100
#SBATCH -C v100-16g


## load Pytorch module
module purge

module load pytorch-gpu/py3/1.10.1

## launch script on every node
set -x

# code execution
srun python main.py --batch_size 2 --lr 0.0001 --n_class 5 \
--dataset_dir /gpfsscratch/idris/sos/ssos022/Medical/BraTS2021/BraTS2021_Training_Data \

Overwriting plug_ai.slurm


In [3]:
%%bash
# submit job
sbatch plug_ai.slurm

Submitted batch job 1077074


In [6]:
from threading import Event
import signal
from IPython.display import clear_output

def quit(signo, _frame):
    print("Interrupted by %d, shutting down" % signo)
    exit.set()

for sig in ('TERM', 'HUP', 'INT'):
    signal.signal(getattr(signal, 'SIG'+sig), quit);

exit = Event()

sq = !squeue -u $USER -n plug_ai
tail = !tail plug_ai.out
while len(sq) >= 2 and not exit.is_set():
    
    clear_output(wait=True)
    print(sq[0], sq[1], sep='\n')
    print(*tail, sep='\n')
    
    sq = !squeue -u $USER -n plug_ai
    tail = !tail plug_ai.out

    exit.wait(1)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1077074   gpu_p13  plug_ai  ssos022 CG       4:10      1 r10i3n6
[Epoch 1/1 |  Step_Epoch 87/626 | Loss 2.1814963817596436]
[Epoch 1/1 |  Step_Epoch 88/626 | Loss 2.1598753929138184]
[Epoch 1/1 |  Step_Epoch 89/626 | Loss 2.164863109588623]
[Epoch 1/1 |  Step_Epoch 90/626 | Loss 2.141308307647705]
[Epoch 1/1 |  Step_Epoch 91/626 | Loss 2.097301721572876]
[Epoch 1/1 |  Step_Epoch 92/626 | Loss 2.113685131072998]
[Epoch 1/1 |  Step_Epoch 93/626 | Loss 2.0842974185943604]
srun: Job step aborted: Waiting up to 62 seconds for job step to finish.
slurmstepd: error: *** STEP 1077074.0 ON r10i3n6 CANCELLED AT 2022-07-11T17:25:57 ***
slurmstepd: error: *** JOB 1077074 ON r10i3n6 CANCELLED AT 2022-07-11T17:25:57 ***


In [5]:
!scancel -u $USER -n plug_ai